In [1]:
!pip install langchain -q
!pip install transformers huggingface-hub -q
!pip install -q bitsandbytes accelerate
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.3 MB/s eta 0:00:00


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model_id = "daryl149/llama-2-7b-chat-hf"

model_load = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
    max_length = 1000
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import pipeline

pipeline = pipeline(
    "text-generation", #task
    model=model_load,
    tokenizer=tokenizer,
    top_k=10,
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [4]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs={"temperature" : 0})

In [5]:
!git clone https://github.com/TheMITTech/shakespeare

Cloning into 'shakespeare'...
remote: Enumerating objects: 2187, done.
remote: Total 2187 (delta 0), reused 0 (delta 0), pack-reused 2187
Receiving objects: 100% (2187/2187), 5.95 MiB | 7.87 MiB/s, done.
Resolving deltas: 100% (1135/1135), done.


Let's collect all the HTML documents into a single directory.

In [6]:
from glob import glob

files = glob("./shakespeare/**/*.html")

In [7]:
import shutil
import os

os.mkdir('./data')
destination_folder = './data/'

for html_file in files:
  shutil.move(html_file, destination_folder + html_file.split("/")[-1])

In [8]:
# install dependencies
!pip install beautifulsoup4 -q

In [9]:
from langchain.document_loaders import BSHTMLLoader, DirectoryLoader

bshtml_dir_loader = DirectoryLoader('./data/', loader_cls=BSHTMLLoader)

data = bshtml_dir_loader.load()

In [10]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=100, chunk_overlap=0, separator="\n")

documents = text_splitter.split_documents(data)

Now that we have our documents split - we need to convert them into embeddings!

True to staying "on-prem" we will be using Hugging Face Hub embedding models to achieve this goal!

Remember, everything we're using today is only an example - feel free to extend/modify wherever you see fit!

In [11]:
# we'll need the sentence transformers library to utilize HF's Sentence XFormer for our embeddings.
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.2 MB/s eta 0:00:00


In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [13]:
# we'll need a few dependencies before we can do this
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━

In [14]:
from langchain.vectorstores import Chroma

persist_directory = "vector_db"

vectordb = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=persist_directory)

In [15]:
doc_retriever = vectordb.as_retriever()

### Final Chain

With that set-up, we're good to set-up our final chain and leverage all the documents we have in our Vector DB!

In [16]:
from langchain.chains import RetrievalQA

shakespeare_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_retriever)

Let's test it out by itself!

In [17]:
shakespeare_qa.run("Could you provide information about the main characters of the novel 'Hamlet'?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (1000) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


" Yes, of course! The main characters of 'Hamlet' are:\n1. Hamlet - The prince of Denmark and the protagonist of the play.\n2. King Claudius - Hamlet's uncle and the king of Denmark. He is the one who murdered Hamlet's father and married Hamlet's mother.\n3. Queen Gertrude - Hamlet's mother and the queen of Denmark. She is the one who marries Claudius after Hamlet's father's death.\n4. Polonius - The lord chamberlain and a close advisor to King Claudius. He is a wise and just man who becomes a victim of Hamlet's madness.\n5. Ophelia - Hamlet's cousin and the daughter of Polonius. She is a sweet and innocent young woman who becomes a victim of Hamlet's madness as well.\n6. Rosencrantz and Guildenstern - Hamlet's childhood friends who return to him as adults. They are loyal to Hamlet but ultimately become victims of his madness.\n7. fortinbras - A Norwegian prince who is mentioned in the play as a potential threat to Denmark. He does not appear in person until the end of the play.\n\nPle

In [18]:
shakespeare_qa.run("Could you summarize the content of the novel 'Hamlet' for me?")

" Yes, of course! Here is a summary of the content of 'Hamlet':\n\nHamlet, Prince of Denmark, returns to Denmark to find his father's ghost, who tells him that he was murdered by his brother, Claudius, who is now the king. Hamlet is tasked with avenging his father's death, but his actions are complicated by his own moral ambiguity and indecision.\nMeanwhile, King Claudius marries Hamlet's mother, Queen Gertrude, and Hamlet's love for Ophelia is complicated by his own feelings of betrayal and anger towards her father, Polonius.\nAs the play progresses, Hamlet's actions become more and more erratic, leading to a bloody confrontation with King Claudius and his accomplices. The play ends with Hamlet's death, as well as the deaths of many others, including King Claudius, Polonius, and Ophelia.\nI hope this summary helps! Let me know if you have any other questions."

### Conclusion

Here we have it!

A system capable of querying over multiple documents - all without every needing to hit an external API!